# Install packages

In [2]:
from IPython.display import clear_output
!pip install alibi[tensorflow]
clear_output()

# Import libraries

In [3]:
# Import necessary modules
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from alibi.explainers import IntegratedGradients
import matplotlib as plt

# Load the pretrained BERT Model
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Render color to explain
from IPython.display import HTML

In [4]:
%whos

Variable                               Type        Data/Info
------------------------------------------------------------
AutoTokenizer                          type        <class 'transformers.mode<...>tion_auto.AutoTokenizer'>
HTML                                   type        <class 'IPython.core.display.HTML'>
IntegratedGradients                    ABCMeta     <class 'alibi.explainers.<...>nts.IntegratedGradients'>
TFAutoModelForSequenceClassification   type        <class 'transformers.mode<...>rSequenceClassification'>
clear_output                           function    <function clear_output at 0x792ccb5c68c0>
np                                     module      <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
pd                                     module      <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
plt                                    module      <module 'matplotlib' from<...>/matplotlib/__init__.py'>
re                                     module      <modu

# Define some functions

In [5]:
def info(obj):
  print(type(obj))
  for item in dir(obj):
    print(item)

In [6]:
# Preprocess and clean the reviews
def preprocess_reviews (reviews):
  # Preprocess the text
  REPLACE_NO_SPACE = re.compile("[.;:,!\'?\"()\[\]]")
  REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

  reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
  reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
  return reviews

In [7]:
def replace_cls_sep(lst, replacement):
    for i in range(len(lst) - 1, -1, -1):
        if lst[i] != 0:
            lst[i] = replacement
            break
    lst[0] = replacement
    return lst

# Tokenize the sentences for the BERT Model
def process_sentences(sentence, tokenizer, max_len):
  # Tokenize the text sentences.
  z = tokenizer(sentence,
                add_special_tokens = False,
                padding = 'do_not_pad', #'max_length', #
                max_length = max_len,
                truncation = True,
                return_token_type_ids = True,
                return_attention_mask = True,
                return_tensors = 'np')
  # replace_cls_sep(z['attention_mask'][0], 0)
  # replace_cls_sep(z['input_ids'][0], 100)
  return z

In [8]:
# Define Wrapper for making sentiment predictions with pretrained BERT Model
class AutoModelWrapper(tf.keras.Model):
  def __init__(self, model_bert, **kwargs):
    super().__init__()
    self.model_bert = model_bert

  def call(self, inputs, attention_mask=None):
    out = self.model_bert(inputs, attention_mask=attention_mask)
    return tf.nn.softmax(out.logits)

  def get_config(self):
    return {}

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [9]:
# Compute hex colors based on the attributions for a single instance.
# Uses a diverging colorscale by default and normalizes and scales
# the colormap so that colors are consistent with the attributions.
def colorize(attrs, cmap='PiYG'):
  cmap_bound = np.abs(attrs).max()
  norm = plt.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
  cmap = plt.cm.get_cmap(cmap)

  # now compute hex values of colors
  colors = list(map(lambda x: plt.colors.rgb2hex(cmap(norm(x))), attrs))
  return colors

# Return HTML markup highlighting text with the desired color.
def hlstr(string, color='white'):
  return f"<mark style=background-color:{color}>{string} </mark>"

In [10]:
def get_blur_baseline(tokenized_text, blur_size = 11):
  padded_array = np.pad(tokenized_text[0], (blur_size//2, blur_size//2), mode='reflect')
  result = np.convolve(padded_array, np.ones(blur_size)/blur_size, mode='valid')

  baseline_blur = np.array([result]).astype(float)
  return baseline_blur

In [11]:
def display_output(text_in_arr, tokenizer, baseline_type, max_len):
  process_test_sample_2 = process_sentences(preprocess_reviews(text_in_arr), tokenizer, max_len)
  x_test_sample_2 = process_test_sample_2['input_ids']
  baseline = x_test_sample_2
  kwargs_2 = {k: tf.constant(v) for k, v in process_test_sample_2.items() if k == 'attention_mask'}
  # probability = auto_model(x_test_sample_2, **kwargs_2).numpy()
  probability = auto_model(x_test_sample_2).numpy()
  predictions_2 = probability.argmax(axis=1)

  if baseline_type == "blur":
    baseline = get_blur_baseline(x_test_sample_2)
  elif baseline_type == "none":
    baseline = None
  else:
    baseline = None

  explanation_2 = ig.explain(
    X               = x_test_sample_2,
    forward_kwargs  = kwargs_2,
    baselines       = baseline,
    target          = predictions_2
  )
  attrs_2 = np.array(explanation_2.attributions[0]).sum(axis=2)

  # Visualize the results
  words_2 = tokenizer.decode(x_test_sample_2[0]).split()
  attrs_2 = np.array(explanation_2.attributions[0])
  attrs_2 = attrs_2.sum(axis=2)
  colors_2 = colorize(attrs_2[0])
  print('-----\nPositive = {}% -> Label {}'.format(probability[0][1]*100, predictions_2))
  return display(HTML("".join(list(map(hlstr, words_2, colors_2)))))

# Setup models and IG

In [12]:
# Call model for getting outputs
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
auto_model_bert = TFAutoModelForSequenceClassification.from_pretrained(model_name)
auto_model_bert.summary()

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [14]:
# 1st parameter for IG function
# Making sentiment predictions with pretrained BERT Model
auto_model = AutoModelWrapper(auto_model_bert)

# 2nd parameter for IG function
# Layer with respect to which the gradients are calculated
block = auto_model.layers[0].layers[0].transformer.layer[0]

# 3rd parameter for IG function
# Method for the integral approximation
method = "gausslegendre"

# 4th parameter for IG function
# Number of step in the path integral approximation from the baseline
n_steps = 32

# 5th parameter for IG function
# Batch size for the internal batching
internal_batch_size = 1

In [15]:
# IG function
ig = IntegratedGradients(
  model               = auto_model,
  layer               = block,
  method              = method,
  n_steps             = n_steps,
  internal_batch_size = internal_batch_size
)

In [16]:
type(auto_model.layers[0].layers[0].embeddings)

transformers.models.distilbert.modeling_tf_distilbert.TFEmbeddings

# Run and test model

In [17]:
max_features = 30000
max_len = 128

# Test sentence for the sentiment analysis
z_first_sample = ['The hotel staff was friendly making our stay exceptionally pleasant.']
z_first_sample = ['The service is quite efficient and reliable. It is a type of online marketplace that allows users to connect with professionals who provide a particular service. The platform is easy to navigate and has a variety of professionals available for hire, from writers to designers to developers. One of the features I appreciated was the ability to filter professionals by their location, hourly rate, and availability. This made it easy to find someone who met my specific needs and budget. The process of hiring a professional was straightforward. Once I found someone I was interested, I was able to communicate with them directly through the platform to discuss the project details and ask any questions I had. The payment process was also very simple and secure']

# Process and tokenize the sentences
z_first_sample = preprocess_reviews(z_first_sample)
print(z_first_sample)
z_first_sample = process_sentences(z_first_sample, tokenizer, max_len)
print(z_first_sample)

['the service is quite efficient and reliable it is a type of online marketplace that allows users to connect with professionals who provide a particular service the platform is easy to navigate and has a variety of professionals available for hire from writers to designers to developers one of the features i appreciated was the ability to filter professionals by their location hourly rate and availability this made it easy to find someone who met my specific needs and budget the process of hiring a professional was straightforward once i found someone i was interested i was able to communicate with them directly through the platform to discuss the project details and ask any questions i had the payment process was also very simple and secure']
{'input_ids': array([[ 1996,  2326,  2003,  3243,  8114,  1998, 10539,  2009,  2003,
         1037,  2828,  1997,  3784, 18086,  2008,  4473,  5198,  2000,
         7532,  2007,  8390,  2040,  3073,  1037,  3327,  2326,  1996,
         4132,  20

In [18]:
tokenizer.convert_ids_to_tokens(z_first_sample['input_ids'][0])

['the',
 'service',
 'is',
 'quite',
 'efficient',
 'and',
 'reliable',
 'it',
 'is',
 'a',
 'type',
 'of',
 'online',
 'marketplace',
 'that',
 'allows',
 'users',
 'to',
 'connect',
 'with',
 'professionals',
 'who',
 'provide',
 'a',
 'particular',
 'service',
 'the',
 'platform',
 'is',
 'easy',
 'to',
 'navigate',
 'and',
 'has',
 'a',
 'variety',
 'of',
 'professionals',
 'available',
 'for',
 'hire',
 'from',
 'writers',
 'to',
 'designers',
 'to',
 'developers',
 'one',
 'of',
 'the',
 'features',
 'i',
 'appreciated',
 'was',
 'the',
 'ability',
 'to',
 'filter',
 'professionals',
 'by',
 'their',
 'location',
 'hourly',
 'rate',
 'and',
 'availability',
 'this',
 'made',
 'it',
 'easy',
 'to',
 'find',
 'someone',
 'who',
 'met',
 'my',
 'specific',
 'needs',
 'and',
 'budget',
 'the',
 'process',
 'of',
 'hiring',
 'a',
 'professional',
 'was',
 'straightforward',
 'once',
 'i',
 'found',
 'someone',
 'i',
 'was',
 'interested',
 'i',
 'was',
 'able',
 'to',
 'communicate',


In [19]:
# 1st parameter for explain function
# Instance for which integrated gradients attribution are computed
x_first_sample = z_first_sample['input_ids']

# 2nd parameter for explain function
# Get tensors for model prediction
kwargs = {k: tf.constant(v) for k, v in z_first_sample.items() if k == 'attention_mask'}

# 4th parameter for explain function
# Get the prediction outputs
# predictions = auto_model_bert.predict
predictions = auto_model(x_first_sample, **kwargs).numpy().argmax(axis=1)

print("Label:", predictions)

Label: [1]


In [20]:
auto_model(x_first_sample, **kwargs).numpy()

array([[0.04296386, 0.9570361 ]], dtype=float32)

# Call IG functions to explain

In [21]:
# # Blur baseline
# blur_size = 11
# padded_array = np.pad(x_first_sample[0], (blur_size//2, blur_size//2), mode='reflect')
# result = np.convolve(padded_array, np.ones(blur_size)/blur_size, mode='valid')
# baseline_blur = np.array([result]).astype(float)
# print(baseline_blur.shape, x_first_sample.shape)

# # Uniform baseline
# baseline_uniform = np.array(tf.random.uniform(shape=x_first_sample.shape, minval = 0, maxval = max_features)).astype(float)
# print(baseline_uniform)

# # Gaussian baseline
# baseline_gaussian = np.array(tf.random.normal(shape=x_first_sample.shape, mean=x_first_sample, stddev=20)).astype(float)
# print(baseline_gaussian)

In [22]:
explanation = ig.explain(
  X               = x_first_sample,
  forward_kwargs  = kwargs, # 1, #
  baselines       = None, # baseline_blur,baseline_uniform,baseline_gaussian,
  target          = predictions
)

# Visualize Results

In [23]:
attrs = np.array(explanation.attributions[0])
attrs = attrs.sum(axis=2)
print('Attributions shape: ', attrs.shape)
words = tokenizer.decode(x_first_sample[0]).split()
colors = colorize(attrs[0])
print('Predicted label = {}'. format(predictions))
HTML("".join(list(map(hlstr, words, colors))))

Attributions shape:  (1, 126)
Predicted label = [1]


<ipython-input-9-7451edd160c0>:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap(cmap)


In [24]:
x = np.array(explanation.attributions[0]).sum(axis=2)[0]
for i in x:
  print(i*1000, ",", sep="", end=" ")

62.332622706890106, -18.360983580350876, 22.523466497659683, 9.402839466929436, 91.04509651660919, 80.15730232000351, 113.78326267004013, 33.858850598335266, 26.200728490948677, 13.220667839050293, -4.096683114767075, 25.82404389977455, 8.909594267606735, 32.077085226774216, 86.16876602172852, 43.18660497665405, 11.267408728599548, 30.042028054594994, 38.62927854061127, 3.9073959924280643, 17.089180648326874, 53.3924363553524, 4.36314195394516, 9.926149621605873, 7.2277807630598545, -10.300733149051666, 27.56519801914692, 9.877919219434261, -74.93538409471512, -13.308274559676647, 2.755405381321907, 15.880463644862175, 13.22475541383028, -9.67045221477747, 0.22367725614458323, 3.0593667179346085, 1.0913857258856297, 13.394726440310478, 0.21588290110230446, 0.03540844772942364, -2.209821715950966, 4.929637536406517, 7.837469689548016, 11.309938505291939, 6.5531800501048565, 7.630491629242897, 8.014209568500519, 3.2732533290982246, -0.6251084851101041, 0.36768504651263356, 9.064424782991

# Quick output

In [25]:
sample = ['I recently had the opportunity to try out a service that left a lasting impression on me. From the moment I entered, the atmosphere was welcoming and comfortable. The staff members were attentive, friendly, and professional. They demonstrated excellent knowledge and expertise in their field, ensuring that every step of the service was executed flawlessly. The attention to detail was remarkable, and I appreciated the careful consideration given to my specific needs and preferences. The service itself was outstanding, providing me with a sense of satisfaction and contentment. I left feeling rejuvenated and grateful for the exceptional experience. I highly recommend this service to anyone seeking a truly memorable experience']

# Process and tokenize the sentences
preprc_sample = preprocess_reviews(sample)
print(preprc_sample)
token_sample = process_sentences(preprc_sample, tokenizer, max_len)
print(token_sample)

x_sample = token_sample['input_ids']
kwargs = {k: tf.constant(v) for k, v in token_sample.items() if k == 'attention_mask'}
predictions = auto_model(x_sample, **kwargs).numpy()
print(predictions[0][1])


['i recently had the opportunity to try out a service that left a lasting impression on me from the moment i entered the atmosphere was welcoming and comfortable the staff members were attentive friendly and professional they demonstrated excellent knowledge and expertise in their field ensuring that every step of the service was executed flawlessly the attention to detail was remarkable and i appreciated the careful consideration given to my specific needs and preferences the service itself was outstanding providing me with a sense of satisfaction and contentment i left feeling rejuvenated and grateful for the exceptional experience i highly recommend this service to anyone seeking a truly memorable experience']
{'input_ids': array([[ 1045,  3728,  2018,  1996,  4495,  2000,  3046,  2041,  1037,
         2326,  2008,  2187,  1037,  9879,  8605,  2006,  2033,  2013,
         1996,  2617,  1045,  3133,  1996,  7224,  2001, 18066,  1998,
         6625,  1996,  3095,  2372,  2020,  2012, 

# Data for testing


In [26]:
import time
from IPython.display import display

extra_single_sentences = [ ["Bad"],
  ["The delivery was late and the product was damaged."],
  ["The customer service at the restaurant was exceptional."],
  ["The airline staff was not helpful during our travel delay."],
  ["I found the tech support to be very responsive and helpful."],
  ["The gym staff is always friendly and supportive."],
  ["The service at the car repair shop was not satisfactory."],
  ["I had a great experience with the customer service team."],
  ["The cleaning service did an excellent job with my apartment."],
  ["The customer support was unable to resolve my issue."],
  ["The hotel provided excellent room service during our stay."],
  ["The medical staff was caring and made the visit less stressful."],
  ["I had a poor experience with the airport security staff."],
  ["The service at the spa was exceptional and relaxing."],
  ["The mechanic was honest and fixed my car efficiently."],
  ["The food delivery was quick and the food was still hot."],
  ["The taxi service was unreliable and the driver was impolite."],
  ["The concert staff ensured a safe and enjoyable experience for everyone."],
  ["The property management service is always responsive to our needs."],
  ["The amusement park staff was very friendly and helpful."],
  ["The delivery service did not handle the package with care."],
  ["The security service at the event was not up to the mark."],
  ["The staff at the post office was very helpful in resolving my issue."],
  ["The customer service at the internet company was very poor."],
  ["The hospital staff provided excellent care during my stay."],
  ["The customer service at the phone company was not able to resolve my issue."],
  ["The technical support was unable to fix the issue with my computer."],
  ["The management at the hotel was not responsive to our complaints."],
  ["The service at the pet store was excellent and the staff was very helpful."],
  ["The staff at the grocery store was not friendly."],
  ["The customer service at the online store was quick to respond and very helpful."]
]

extra_double_sentences = [ ["Good"],
  ["Although the food was delicious, the service was slow and unresponsive."],
  ["The hotel was clean and comfortable, but the staff was rude and unhelpful."],
  ["Despite the high prices, the quality of the products was excellent."],
  ["The customer service representative was friendly and knowledgeable, but the wait time on the phone was too long."],
  ["Although the location was convenient, the room was small and cramped."],
  ["The website was easy to navigate, but the checkout process was confusing and frustrating."],
  ["Despite the crowded atmosphere, the bartender was attentive and made great drinks."],
  ["The prices were reasonable, but the portion sizes were disappointingly small."],
  ["Although the store was busy, the sales associate was patient and helpful."],
  ["The spa was relaxing and rejuvenating, but the prices were a bit steep."],
  ["Despite the long wait time, the doctor was thorough and attentive during the appointment."],
  ["Although the delivery was fast, the food was cold and not as described."],
  ["The gym was well-equipped and clean, but the music was too loud and distracting."],
  ["Despite the limited menu options, the food was flavorful and well-prepared."],
  ["Although the staff was friendly, the hotel room was not as clean as expected."],
  ["The customer service was prompt and efficient, but the product was not as advertised."],
  ["Despite the high price tag, the quality of the service was worth it."],
  ["Although the restaurant was busy, the hostess was able to find us a table quickly."],
  ["The store had a great selection of products, but the checkout line was long and slow-moving."],
  ["Despite the inclement weather, the tour guide was knowledgeable and engaging."],
  ["Although the hotel was in a great location, the noise level was too high to get a good night's sleep."],
  ["The coffee shop had a cozy atmosphere, but the coffee itself was not very good."],
  ["Despite the friendly staff, the food took a long time to arrive at the table."],
  ["Although the website had a lot of information, it was difficult to find what I was looking for."],
  ["The customer service team was able to resolve my issue quickly, but the problem should not have occurred in the first place."],
  ["Despite the low prices, the quality of the products was surprisingly good."],
  ["Although the restaurant had a unique atmosphere, the food was not very flavorful."],
  ["The airline had comfortable seats and good in-flight entertainment, but the food was mediocre."],
  ["Despite the long wait time, the doctor was able to diagnose my condition accurately."],
  ["Although the hotel room was spacious, the air conditioning was not working properly."]
]

extra_long_sentence = ['I recently had the pleasure of experiencing a service that truly exceeded my expectations. From the moment I entered, the atmosphere was inviting and comfortable. The attentive staff provided exceptional care and service, displaying professionalism and genuine dedication. The attention to detail was evident in every aspect, ensuring a seamless and satisfying experience. The quality of the service was unparalleled, leaving me feeling impressed and thoroughly pleased. I appreciated the personalized approach, tailored to meet my specific needs and preferences. Overall, this service delivered an exceptional and gratifying experience that I highly recommend to anyone seeking awesome service and utmost satisfaction.']



# Extra test

In [27]:
display_output(extra_long_sentence, tokenizer, "none", max_len)

-----
Positive = 99.97265934944153% -> Label [1]


<ipython-input-9-7451edd160c0>:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap(cmap)


In [ ]:
abc

In [29]:
for z_test_sample_2 in extra_single_sentences:
  # display_output(z_test_sample_2, tokenizer, "blur", max_len)
  display_output(z_test_sample_2, tokenizer, "none", max_len)

-----
Positive = 14.14966732263565% -> Label [0]


<ipython-input-9-7451edd160c0>:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap(cmap)


-----
Positive = 0.07577495998702943% -> Label [0]


-----
Positive = 99.95248317718506% -> Label [1]


-----
Positive = 0.05651916726492345% -> Label [0]


...

-----
Positive = 0.026040829834528267% -> Label [0]


-----
Positive = 0.032020913204178214% -> Label [0]


-----
Positive = 99.91828799247742% -> Label [1]


-----
Positive = 0.0324438966345042% -> Label [0]


-----
Positive = 98.13352823257446% -> Label [1]


In [ ]:
abc

In [30]:
for z_test_sample_2 in extra_double_sentences:
  # display_output(z_test_sample_2, tokenizer, "blur", max_len)
  display_output(z_test_sample_2, tokenizer, "none", max_len)

-----
Positive = 97.50707745552063% -> Label [1]


<ipython-input-9-7451edd160c0>:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap(cmap)


-----
Positive = 99.44288730621338% -> Label [1]


-----
Positive = 0.6475139409303665% -> Label [0]


-----
Positive = 99.15306568145752% -> Label [1]


...

-----
Positive = 99.76171851158142% -> Label [1]


-----
Positive = 0.40541375055909157% -> Label [0]


-----
Positive = 0.5059498827904463% -> Label [0]


-----
Positive = 0.0709100451786071% -> Label [0]


-----
Positive = 0.02251124387839809% -> Label [0]
